In [1]:
import numpy as np
from recstate2mode import UW_,UV_,C_,mu_,Sigma_,G_matrix

In [13]:
gamma = np.array([0,0],dtype = np.complex128)
phi = np.array([0,0],dtype = np.complex128)
zeta = np.array([0,0],dtype = np.complex128)
theta = 0
psi = 0
theta1 = 0.3
psi1 = 0
cutoff_dim = 2
cutoff= 2
cutoff_plus = 10


In [14]:
old_state = np.random.rand(cutoff_dim,cutoff_dim) + 1.0j*np.random.rand(cutoff_dim,cutoff_dim)
old_state  /= np.linalg.norm(old_state)

In [112]:
W = UW_(phi,theta1,psi1)
V = UV_(theta,psi)
    
C = C_(gamma, W, zeta, V)
mu = mu_(gamma, W, zeta, V)
Sigma = Sigma_(W, zeta, V)

cutoff = old_state.shape[0]
dtype = old_state.dtype


sqrt = np.sqrt(np.arange(cutoff, dtype=dtype))
sqrt_col = np.sqrt(np.arange(cutoff, dtype=dtype)).reshape(cutoff,1)
print(sqrt_col)
sqrts = np.zeros((cutoff, cutoff), dtype=dtype)
for m in range(0,cutoff):
    for n in range(0,cutoff):
        sqrts[m,n]= np.sqrt(m+1)*np.sqrt(n+1)

print(sqrts)
G_00pq = np.zeros((cutoff, cutoff), dtype=dtype)


G_00pq[0,0] = C
for q in range(1, cutoff):
    G_00pq[0,q] = (mu[3]*G_00pq[0,q-1] - Sigma[3,3]*sqrt[q-1]*G_00pq[0,q-2])/sqrt[q]


for p in range(1,cutoff):
    for q in range(0,cutoff):
        G_00pq[p,q] = (mu[2]*G_00pq[p-1,q] - Sigma[2,2]*sqrt[p-1]*G_00pq[p-2,q] - Sigma[2,3]*sqrt[q]*G_00pq[p-1,q-1])/sqrt[p]

[[0.+0.j]
 [1.+0.j]]
[[1.        +0.j 1.41421356+0.j]
 [1.41421356+0.j 2.        +0.j]]


In [16]:
Sigma

array([[ 0.        +0.j,  0.        +0.j, -0.95533649+0.j,
         0.29552021+0.j],
       [ 0.        +0.j,  0.        +0.j, -0.29552021+0.j,
        -0.95533649+0.j],
       [-0.95533649+0.j, -0.29552021+0.j,  0.        +0.j,
         0.        +0.j],
       [ 0.29552021+0.j, -0.95533649+0.j,  0.        +0.j,
         0.        +0.j]])

In [17]:
G_00pq

array([[1.+0.j, 0.-0.j],
       [0.-0.j, 0.+0.j]])

In [125]:
# R_00^jk = G_00pq * a^j b^k|old_state>

R = np.zeros((cutoff, cutoff, cutoff+1, cutoff+1), dtype=dtype)

R[0,0,0,0] = np.sum(G_00pq * old_state)

for k in range(1,cutoff):
    R[0,0,0,k]=np.sum(G_00pq[:,k:] * (old_state[:,:cutoff-k] * sqrt_col[:cutoff-k] ))
    
for j in range(1,cutoff):
    R[0,0,j,0]=np.sum(G_00pq[j:,:] * (old_state[:cutoff-j,:] * sqrt[:cutoff-j] ))
    
for k in range(1,cutoff):
    for j in range(1,cutoff): #[:cutoff-j,:cutoff-k] or [j:,k:]
        R[0,0,j,k] = np.sum(G_00pq[:cutoff-j,:cutoff-k] *sqrts[:cutoff-j,:cutoff-k] * old_state[j:,k:] )
        
#R_0n^jk
for k in range(0,cutoff):
    for j in range(0,cutoff):
        for n in range(1,cutoff):
            R[0,n,j,k] = mu[1]/sqrt[n]*R[0,n-1,j,k] - Sigma[1,1]/sqrt[n]*sqrt[n-1]*R[0,n-2,j,k] - Sigma[1,2]/sqrt[n]*R[0,n-1,j+1,k] - Sigma[1,3]/sqrt[n]*R[0,n-1,j,k+1]


for k in range(0,cutoff):
    for j in range(0,cutoff):
        for n in range(0,cutoff):
            for m in range(1,cutoff):
                R[m,n,j,k] = mu[0]/sqrt[m]*R[m-1,n,j,k] - Sigma[0,0]/sqrt[m]*sqrt[m-1]*R[m-2,n,j,k] - Sigma[0,1]*sqrt[n]/sqrt[m]*R[m-1,n-1,j,k] - Sigma[0,2]/sqrt[m]*R[m-1,n,j+1,k] - Sigma[0,3]/sqrt[m]*R[m-1,n,j,k+1]
                
                
print("R",R[:,:,:cutoff,:cutoff])

R [[[[ 0.1200163 +0.07489342j  0.        +0.j        ]
   [ 0.        +0.j          0.36884203+0.34609239j]]

  [[ 0.        +0.j          0.10900027+0.1022773j ]
   [ 0.35236825+0.33063469j  0.        +0.j        ]]]


 [[[ 0.        +0.j          0.35236825+0.33063469j]
   [-0.10900027-0.1022773j   0.        +0.j        ]]

  [[ 0.30441847+0.28564238j  0.        +0.j        ]
   [ 0.        +0.j          0.        +0.j        ]]]]


In [126]:
R[:,:,0,0]

array([[0.1200163 +0.07489342j, 0.        +0.j        ],
       [0.        +0.j        , 0.30441847+0.28564238j]])

In [127]:
G = G_matrix(gamma, phi, theta1, psi1, zeta, theta, psi,cutoff)

In [128]:
np.einsum("ab,abcd->cd",old_state,G)

array([[0.1200163 +0.07489342j, 0.076213  +0.1698001j ],
       [0.33633466+0.75921089j, 0.30441847+0.28564238j]])